Test: Animation with multiple Paths - https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/


In [4]:
# Installieren der notwendigen Module
import piplite
await piplite.install("pandas")
await piplite.install("folium")
await piplite.install("geojson")

import pandas as pd
import datetime as dt
import folium
import os
from branca.element import Figure
from folium import plugins
from random import randint

df = pd.DataFrame({}) # leerer DataFrame, welcher später mit Daten gefüllt wird

# Interation über alle Bangle-Dateien
for file in os.listdir("bangle_files"):
    if file.endswith(".csv"):

        # Einlesen der Daten
        df_new = pd.read_csv(os.path.join("bangle_files", file))
        df_new.head()
        df_new.columns = ['Time', 'latitude',
                        'longitude', 'color']  # Spaltennamen anpassen
        color = '#%06X' % randint(0, 0xFFFFFF)
        df_new['color'] = df_new['color'].apply(lambda x: color)
        #df_new['fileName'] = df_new['fileName'].apply(lambda x: "red")
        df_new.head()
        df = pd.concat([df, df_new ])  # Dateien in einen einzigen Dataframe zusammenfügen

# Nur Zeilen behalten, welche auch GPS-Daten enthalten
df = df[df['longitude'].notna()]
df['Time'] = df['Time'].apply(lambda x: pd.to_datetime(dt.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')))  # Datumsformat anpassen (Pandas Datetime)

# Neue Spalten: Stunde & Minute, um damit später mehrere Tabellen zu joinen
df['hour'] = df['Time'].apply(lambda x: x.hour+1)
df['minute'] = df['Time'].apply(lambda x: x.minute+1)

df['Time'] = df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
#df = df.drop(columns=['Time'])

df = df[df.index % 6 == 0]  # Selects every 6th raw starting from 0. So we drop 90% of rows to get a faster performance. 1 row per Minute left

# Data-Frame in geojson umwandeln, sodass dieser dann mit Folium dargestellt werden kann
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):

    geojson = {'type': 'FeatureCollection', 'features': []}

    # Durch jede einzelne Zeile des Dataframes iterieren
    row_iterator = df.iterrows()
    tempTime = ""
    tempLon = ""
    tempLat = ""
    
    for _, row in row_iterator:
        if (tempTime != ""):
            # jeweils die notwendigen Daten in das geojson-Format einfügen
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [],
                },
                "properties": {
                    "times": [tempTime, row['Time']],
                    "style": {
                        "color": row['color'],
                        "weight":  5,
                    },
                },
            }
            feature['geometry']['coordinates'] = [[tempLon, tempLat], [row[lon], row[lat]],]
           
            # Alle Zusätzlichen Spalten als zusätzlich Feature in geojson einfügen
            # for prop in properties:
            #    feature['properties'][prop] = row[prop]
            geojson['features'].append(feature)
        tempLon = row[lon]
        tempLat = row[lat]
        tempTime = row['Time']
    return geojson


df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
cols = ['latitude', 'longitude', 'color', 'hour', 'minute']
geojson = df_to_geojson(df, cols)

# Folium-Map erstellen
fig = Figure(height=400, width=750)
m = folium.Map(location=[50.912938, 8.024302], zoom_start=16)
plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": geojson['features'],
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)
#TimestampedGeoJson(geojson).add_to(m)
fig.add_child(m)
